<a href="https://colab.research.google.com/github/yashaswini1764/NLP/blob/main/NLP_PROJECT_1764.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers gensim tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 23.7 MB/s eta 0:00:00


In [25]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Define stopwords
stop = set(stopwords.words('english'))

# Define text cleaning function
def clean_text(t):
    t = re.sub('[^a-zA-Z]', ' ', t)  # remove non-letter characters
    t = t.lower()                    # lowercase text
    t = t.split()                    # tokenize
    t = [w for w in t if w not in stop]  # remove stopwords
    return " ".join(t)

# Load the data into a DataFrame
df = pd.read_csv('/content/Restaurant reviews.csv')

# Ensure Review column is string
df['Review'] = df['Review'].astype(str)

# Apply cleaning
df['clean'] = df['Review'].apply(clean_text)

# Show first few rows
print(df.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


        Restaurant              Reviewer  \
0  Beyond Flavours     Rusha Chakraborty   
1  Beyond Flavours  Anusha Tirumalaneedi   
2  Beyond Flavours       Ashok Shekhawat   
3  Beyond Flavours        Swapnil Sarkar   
4  Beyond Flavours                Dileep   

                                              Review Rating  \
0  The ambience was good, food was quite good . h...      5   
1  Ambience is too good for a pleasant evening. S...      5   
2  A must try.. great food great ambience. Thnx f...      5   
3  Soumen das and Arun was a great guy. Only beca...      5   
4  Food is good.we ordered Kodi drumsticks and ba...      5   

                  Metadata             Time  Pictures    7514  \
0   1 Review , 2 Followers  5/25/2019 15:54         0  2447.0   
1  3 Reviews , 2 Followers  5/25/2019 14:20         0     NaN   
2  2 Reviews , 3 Followers  5/24/2019 22:54         0     NaN   
3    1 Review , 1 Follower  5/24/2019 22:11         0     NaN   
4  3 Reviews , 2 Followers  5/2

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

# Drop rows where 'Rating' is NaN
df_cleaned = df.dropna(subset=['Rating']).copy()

X = df_cleaned['clean']
y = df_cleaned['Rating'] # Corrected column name to 'Rating'

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_tfidf = SVC(kernel='linear')
model_tfidf.fit(X_train_tfidf,y_train)
pred = model_tfidf.predict(X_test_tfidf)

print("TF-IDF Accuracy:",accuracy_score(y_test,pred))
print(classification_report(y_test,pred))

TF-IDF Accuracy: 0.6231811339688911
              precision    recall  f1-score   support

           1       0.67      0.83      0.74       345
         1.5       0.00      0.00      0.00         2
           2       0.41      0.16      0.22       129
         2.5       0.00      0.00      0.00         5
           3       0.51      0.33      0.40       252
         3.5       0.00      0.00      0.00         5
           4       0.50      0.48      0.49       486
         4.5       0.00      0.00      0.00        10
           5       0.70      0.81      0.75       758
        Like       0.00      0.00      0.00         1

    accuracy                           0.62      1993
   macro avg       0.28      0.26      0.26      1993
weighted avg       0.59      0.62      0.60      1993



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
from gensim.models import Word2Vec

sentences = [i.split() for i in df['clean']]
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean'])

X_seq = tokenizer.texts_to_sequences(df['clean'])
maxlen = 100
X_pad = pad_sequences(X_seq, maxlen=maxlen)

y = df['Rating'] # Corrected column name to 'Rating'

X_train,X_test,y_train,y_test = train_test_split(X_pad,y,test_size=0.2,random_state=42)

word2vec-lstm

In [9]:
import numpy as np

vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size,100))

for word, idx in tokenizer.word_index.items():
    if word in w2v.wv:
        embedding_matrix[idx] = w2v.wv[word]

In [10]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical # Removed unnecessary import
import numpy as np
import pandas as pd # Import pandas for get_dummies

# One-hot encode the target variable
# Ensure y is defined correctly from previous steps, assuming y is from df['Rating']
y_encoded = pd.get_dummies(y).values

# Split data again after encoding y
# Make sure X_pad is defined from previous steps
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)


model_w2v = Sequential([
    Embedding(vocab_size,100,weights=[embedding_matrix],input_length=maxlen,trainable=False),
    Bidirectional(LSTM(128)),
    Dense(y_encoded.shape[1], activation='softmax') # Changed to match the number of unique classes after one-hot encoding
])

model_w2v.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_w2v.summary()

model_w2v.fit(X_train,y_train_encoded,epochs=5,batch_size=32,validation_data=(X_test,y_test_encoded))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     1,448,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,448,400 (5.53 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,448,400 (5.53 MB)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 66s 253ms/step - accuracy: 0.4456 - loss: 1.4592 - val_accuracy: 0.5260 - val_loss: 1.1857
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 68s 272ms/step - accuracy: 0.5482 - loss: 1.1642 - val_accuracy: 0.5185 - val_loss: 1.1651
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 64s 255ms/step - accuracy: 0.5500 - loss: 1.1268 - val_accuracy: 0.5640 - val_loss: 1.0842
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 63s 253ms/step - accuracy: 0.5718 - loss: 1.0757 - val_accuracy: 0.5825 - val_loss: 1.0401
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 80s 246ms/step - accuracy: 0.5985 - loss: 1.0299 - val_accuracy: 0.5720 - val_loss: 1.0853


bert-to-bert

In [11]:
from transformers import BertTokenizer, TFBertModel

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Al

In [12]:
def bert_encode(texts, tokenizer, max_len=100):
    input_ids, att = [], []

    for t in texts:
        enc = tokenizer.encode_plus(t, max_length=max_len, truncation=True, padding='max_length')
        input_ids.append(enc['input_ids'])
        att.append(enc['attention_mask'])
    return np.array(input_ids), np.array(att)

X_input_ids, X_att = bert_encode(df['clean'], tokenizer_bert)

X_train_ids, X_test_ids, X_train_att, X_test_att, y_train, y_test = train_test_split(
    X_input_ids, X_att, y, test_size=0.2, random_state=42)

In [13]:
def evaluate_classification(y_true, y_pred, labels=None):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, sup = precision_recall_fscore_support(y_true, y_pred, average='weighted', zero_division=0)
    report = classification_report(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision (weighted): {prec:.4f}")
    print(f"Recall (weighted): {rec:.4f}")
    print(f"F1-score (weighted): {f1:.4f}\n")
    print("Classification report:\n", report)
    print("Confusion matrix:\n", cm)
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'report': report, 'confusion_matrix': cm}


precision,recall,fn score,confusion matrix

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support, confusion_matrix
import numpy as np
import pandas as pd # Ensure pandas is imported

# Re-establish X and y based on df_cleaned to ensure correct splits for evaluation
X_data = df_cleaned['clean']
y_data = df_cleaned['Rating']

# Re-split the data to ensure correct X_train, X_test, y_train, y_test for TF-IDF evaluation
X_train_tfidf_eval, X_test_tfidf_eval, y_train_tfidf_eval, y_test_tfidf_eval = train_test_split(
    X_data, y_data, test_size=0.2, random_state=42
)

# Re-initialize and fit TF-IDF vectorizer for this evaluation block
tfidf_for_eval = TfidfVectorizer(max_features=5000)
X_train_tfidf_processed = tfidf_for_eval.fit_transform(X_train_tfidf_eval)
X_test_tfidf_processed = tfidf_for_eval.transform(X_test_tfidf_eval)

# Define num_classes using the unique values from the original y_data
num_classes = len(y_data.unique())

print('\n---- TF-IDF + SVM ----')
# Re-initialize and train the SVC model with the correct data
model_tfidf_for_eval = SVC(kernel='linear')
model_tfidf_for_eval.fit(X_train_tfidf_processed, y_train_tfidf_eval)
svm_pred = model_tfidf_for_eval.predict(X_test_tfidf_processed)

# Evaluate with the correct y_test and predictions
evaluate_classification(y_test_tfidf_eval, svm_pred, labels=np.sort(y_data.unique()))


---- TF-IDF + SVM ----
Accuracy: 0.6232
Precision (weighted): 0.5947
Recall (weighted): 0.6232
F1-score (weighted): 0.5999

Classification report:
               precision    recall  f1-score   support

           1       0.67      0.83      0.74       345
         1.5       0.00      0.00      0.00         2
           2       0.41      0.16      0.22       129
         2.5       0.00      0.00      0.00         5
           3       0.51      0.33      0.40       252
         3.5       0.00      0.00      0.00         5
           4       0.50      0.48      0.49       486
         4.5       0.00      0.00      0.00        10
           5       0.70      0.81      0.75       758
        Like       0.00      0.00      0.00         1

    accuracy                           0.62      1993
   macro avg       0.28      0.26      0.26      1993
weighted avg       0.59      0.62      0.60      1993

Confusion matrix:
 [[288   0  12   0  15   0  15   0  15   0]
 [  2   0   0   0   0   0   0 

{'accuracy': 0.6231811339688911,
 'precision': 0.5947489954405277,
 'recall': 0.6231811339688911,
 'f1': 0.5998889227763868,
 'report': '              precision    recall  f1-score   support\n\n           1       0.67      0.83      0.74       345\n         1.5       0.00      0.00      0.00         2\n           2       0.41      0.16      0.22       129\n         2.5       0.00      0.00      0.00         5\n           3       0.51      0.33      0.40       252\n         3.5       0.00      0.00      0.00         5\n           4       0.50      0.48      0.49       486\n         4.5       0.00      0.00      0.00        10\n           5       0.70      0.81      0.75       758\n        Like       0.00      0.00      0.00         1\n\n    accuracy                           0.62      1993\n   macro avg       0.28      0.26      0.26      1993\nweighted avg       0.59      0.62      0.60      1993\n',
 'confusion_matrix': array([[288,   0,  12,   0,  15,   0,  15,   0,  15,   0],
      

glove-lstm

In [15]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip


--2025-11-18 09:25:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-11-18 09:25:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-11-18 09:25:31--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [16]:
embedding_index = {}

with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = vector

In [17]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# ----------------------------------------------------
# 1. Example dataset
# ----------------------------------------------------
texts = [
    "I love this movie",
    "This film is terrible",
    "Absolutely fantastic movie",
    "Worst movie ever",
    "I really enjoyed this",
    "I hate this film"
]

labels = [1, 0, 1, 0, 1, 0]   # 1 = positive, 0 = negative

# ----------------------------------------------------
# 2. Tokenization
# ----------------------------------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=10)
y = np.array(labels)

# ----------------------------------------------------
# 3. Train-test split
# ----------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------------------------------


lstm

In [18]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# -----------------------------------------------------
# 1. Sample dataset
# -----------------------------------------------------
texts = [
    "I love this movie",
    "This movie is terrible",
    "Absolutely fantastic film",
    "Worst movie ever",
    "I enjoyed this a lot",
    "I hate this movie"
]

labels = [1, 0, 1, 0, 1, 0]

# -----------------------------------------------------
# 2. Tokenize
# -----------------------------------------------------
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=10)
y = np.array(labels)

# -----------------------------------------------------
# 3. Train-Test Split
# -----------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -----------------------------------------------------
# 4. Build Simple LSTM Model
# -----------------------------------------------------
model = Sequential([
    Embedding(vocab_size, 64, input_length=10),
    LSTM(128),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()

# -----------------------------------------------------
# 5. Train the Model
# -----------------------------------------------------
model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=2,
    validation_split=0.2,
    verbose=1
)

# -----------------------------------------------------
# 6. Evaluate
# -----------------------------------------------------
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", acc)

# -----------------------------------------------------
# 7. Predict
# -----------------------------------------------------
sample_text = ["I really liked this film"]
seq = tokenizer.texts_to_sequences(sample_text)
pad = pad_sequences(seq, maxlen=10)

pred = model.predict(pad)[0][0]

print("Prediction Score:", pred)
print("Sentiment:", "Positive" if pred > 0.5 else "Negative")


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 325ms/step - accuracy: 0.7778 - loss: 0.6894 - val_accuracy: 0.0000e+00 - val_loss: 0.7134
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.7778 - loss: 0.6795 - val_accuracy: 0.0000e+00 - val_loss: 0.7181
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.6111 - loss: 0.6768 - val_accuracy: 0.0000e+00 - val_loss: 0.7213
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.7778 - loss: 0.6648 - val_accuracy: 0.0000e+00 - val_loss: 0.7314
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.6111 - loss: 0.6690 - val_accuracy: 0.0000e+00 - val_loss: 0.7384
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - accuracy: 0.7778 - loss: 0.6512 - val_accuracy: 0.0000e+00 - val_loss: 0.7499
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.7778 - loss: 0.6236 - val_accuracy: 0.0000e+00 - val_loss: 0.7574
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6111 - loss: 0.6560 - val_a

bert lstm

In [19]:
pip install transformers torch datasets


In [20]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel, Trainer, TrainingArguments
from datasets import Dataset


In [21]:
texts = [
    "This movie is great!",
    "Terrible film",
    "I loved the acting",
    "Worst movie ever made"
]

labels = [1, 0, 1, 0]

dataset = Dataset.from_dict({"text": texts, "label": labels})


In [22]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=64
    )

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type="torch",
                   columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

In [23]:
class BertLSTM(nn.Module):
    def __init__(self, hidden_size=256, num_classes=2):
        super().__init__()

        # 1. Pretrained BERT (frozen or trainable)
        self.bert = BertModel.from_pretrained("bert-base-uncased")

        # 2. LSTM layer
        self.lstm = nn.LSTM(
            input_size=768,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        # 3. Classification head
        self.classifier = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, input_ids, attention_mask, labels=None):

        # BERT output
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        sequence_output = outputs.last_hidden_state   # (B, seq, 768)

        # LSTM
        lstm_out, _ = self.lstm(sequence_output)  # (B, seq, hidden*2)

        # Use CLS token from LSTM output
        cls_out = lstm_out[:, 0, :]  # (B, hidden*2)

        logits = self.classifier(cls_out)

        # Compute loss
        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {"loss": loss, "logits": logits}


In [34]:
model = BertLSTM()

training_args = TrainingArguments(
    output_dir="bert2bert",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_steps=10,
    report_to='none' # Explicitly disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=3, training_loss=0.6726484298706055, metrics={'train_runtime': 58.6825, 'train_samples_per_second': 0.204, 'train_steps_per_second': 0.051, 'total_flos': 0.0, 'train_loss': 0.6726484298706055, 'epoch': 3.0})

In [35]:
sentence = "I enjoyed the movie!"

tokens = tokenizer(sentence, return_tensors="pt")
output = model(tokens["input_ids"], tokens["attention_mask"])
pred = torch.argmax(output["logits"]).item()

print("Sentiment:", "Positive" if pred == 1 else "Negative")


Sentiment: Positive
